In [1]:
import os, shutil
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
from skimage.io import imread
from keras import models
from keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_folder = 'chest_xray/train'
test_folder = 'chest_xray/val'
val_folder = 'chest_xray/test'

In [123]:
train_n_folder = 'chest_xray/chest_xray/train/NORMAL'
train_p_folder = 'chest_xray/chest_xray/train/PNEUMONIA'
test_n_folder = 'chest_xray/chest_xray/val/NORMAL'
test_p_folder = 'chest_xray/chest_xray/val/PNEUMONIA'
val_n_folder = 'chest_xray/chest_xray/test/NORMAL'
val_p_folder = 'chest_xray/chest_xray/test/PNEUMONIA'

Store all image names

In [4]:
train_n = [file for file in os.listdir(train_n_folder) if file.endswith('.jpeg')]
train_p = [file for file in os.listdir(train_p_folder) if file.endswith('.jpeg')]

In [5]:
test_n = [file for file in os.listdir(test_n_folder) if file.endswith('.jpeg')]
test_p = [file for file in os.listdir(test_p_folder) if file.endswith('.jpeg')]

In [6]:
val_n = [file for file in os.listdir(val_n_folder) if file.endswith('.jpeg')]
val_p = [file for file in os.listdir(val_p_folder) if file.endswith('.jpeg')]

Image Processing

In [7]:
len(train_n) + len(train_p)

5216

In [8]:
# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_folder, 
        target_size=(64, 64), batch_size= 5216)

Found 5216 images belonging to 2 classes.


In [9]:
len(test_n) + len(test_p)

16

In [10]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), batch_size = 16)

Found 16 images belonging to 2 classes.


In [11]:
len(val_n) + len(val_p)

624

In [12]:
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder, 
        target_size=(64, 64), batch_size = 624) 

Found 624 images belonging to 2 classes.


Create the datasets

In [13]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [93]:
train_labels

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

reshape

In [54]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

In [55]:
print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 12288)
(16, 12288)
(624, 12288)


In [56]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (16,1))
val_y = np.reshape(val_labels[:,0], (624,1))

First Model

In [57]:
first_model = models.Sequential()
first_model.add(layers.Dense(20, activation='relu', input_shape=(12288,)))
first_model.add(layers.Dense(1, activation='sigmoid'))

In [58]:
first_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [59]:
first_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 20)                245780    
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 21        
Total params: 245,801
Trainable params: 245,801
Non-trainable params: 0
_________________________________________________________________


In [60]:
result_first = first_model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
163/163 [==============================] - 1s 4ms/step - loss: 0.5688 - recall_5: 0.0067 - val_loss: 0.5991 - val_recall_5: 0.0000e+00
Epoch 2/50
163/163 [==============================] - 1s 3ms/step - loss: 0.3857 - recall_5: 0.0902 - val_loss: 0.6123 - val_recall_5: 0.0983
Epoch 3/50
163/163 [==============================] - 0s 3ms/step - loss: 0.3093 - recall_5: 0.6547 - val_loss: 0.6150 - val_recall_5: 0.1795
Epoch 4/50
163/163 [==============================] - 0s 3ms/step - loss: 0.2527 - recall_5: 0.7405 - val_loss: 0.4968 - val_recall_5: 0.4060
Epoch 5/50
163/163 [==============================] - 1s 3ms/step - loss: 0.2223 - recall_5: 0.7912 - val_loss: 0.5301 - val_recall_5: 0.4060
Epoch 6/50
163/163 [==============================] - 1s 3ms/step - loss: 0.1731 - recall_5: 0.8449 - val_loss: 0.7480 - val_recall_5: 0.2692
Epoch 7/50
163/163 [==============================] - 1s 3ms/step - loss: 0.1661 - recall_5: 0.8486 - val_loss: 0.4917 - val_recall_5: 0.4872
Ep

Visualize Loss and Recall Score

In [61]:
first_results_train = first_model.evaluate(train_img, train_y)
first_results_val = first_model.evaluate(val_img, val_y)
print(first_results_train)
print(first_results_val)

20/20 [==============================] - 0s 2ms/step - loss: 1.0560 - recall_5: 0.3077
[0.07539720833301544, 0.9276659488677979]
[1.0559560060501099, 0.3076923191547394]


First Model Plus Class Weight

In [62]:
weight_for_0 = 3875/1341
weight_for_1 = 1

class_weight = {0: (3875/1341), 1: 1}

print(weight_for_0)
print(weight_for_1)

2.889634601043997
1


In [63]:
cb_first_model = models.Sequential()
cb_first_model.add(layers.Dense(20, activation='relu', input_shape=(12288,)))
cb_first_model.add(layers.Dense(1, activation='sigmoid'))

In [64]:
cb_first_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [65]:
result_cb_first = cb_first_model.fit(train_img,
                    train_y,
                    epochs=50,
                    class_weight = class_weight,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
163/163 [==============================] - 1s 5ms/step - loss: 0.9519 - recall_6: 0.0037 - val_loss: 0.7100 - val_recall_6: 0.0000e+00
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.5344 - recall_6: 0.1939 - val_loss: 0.5311 - val_recall_6: 0.0171
Epoch 3/50
163/163 [==============================] - 1s 3ms/step - loss: 0.4223 - recall_6: 0.4795 - val_loss: 0.7675 - val_recall_6: 0.1111
Epoch 4/50
163/163 [==============================] - 1s 3ms/step - loss: 0.3830 - recall_6: 0.5913 - val_loss: 0.7547 - val_recall_6: 0.2222
Epoch 5/50
163/163 [==============================] - 1s 3ms/step - loss: 0.3322 - recall_6: 0.6950 - val_loss: 1.9940 - val_recall_6: 0.0085
Epoch 6/50
163/163 [==============================] - 1s 3ms/step - loss: 0.3381 - recall_6: 0.7122 - val_loss: 0.8241 - val_recall_6: 0.2521
Epoch 7/50
163/163 [==============================] - 1s 3ms/step - loss: 0.3091 - recall_6: 0.7353 - val_loss: 1.2096 - val_recall_6: 0.1496
Ep

In [66]:
cb_first_results_train = cb_first_model.evaluate(train_img, train_y)
cb_first_results_val = cb_first_model.evaluate(val_img, val_y)

print(first_results_train)
print(first_results_val)
print(cb_first_results_train)
print(cb_first_results_val)

20/20 [==============================] - 0s 2ms/step - loss: 0.8246 - recall_6: 0.4786
[0.07539720833301544, 0.9276659488677979]
[1.0559560060501099, 0.3076923191547394]
[0.09835421293973923, 0.9753915071487427]
[0.8245831727981567, 0.47863247990608215]


add Early Stopping

In [67]:
# Import EarlyStopping and ModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the callbacks
early_stopping = [EarlyStopping(monitor='val_loss', patience=10), 
                  ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [68]:
es_cb_first_model = models.Sequential()
es_cb_first_model.add(layers.Dense(20, activation='relu', input_shape=(12288,)))
es_cb_first_model.add(layers.Dense(1, activation='sigmoid'))

In [69]:
es_cb_first_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [70]:
result_es_cb_first = es_cb_first_model.fit(train_img,
                    train_y,
                    epochs=50,
                    class_weight = class_weight,
                    callbacks=early_stopping,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6843 - recall_7: 0.0321 - val_loss: 0.7372 - val_recall_7: 0.0171
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.4532 - recall_7: 0.4526 - val_loss: 1.6991 - val_recall_7: 0.0000e+00
Epoch 3/50
163/163 [==============================] - 1s 5ms/step - loss: 0.4126 - recall_7: 0.6070 - val_loss: 2.6199 - val_recall_7: 0.0000e+00
Epoch 4/50
163/163 [==============================] - 1s 3ms/step - loss: 0.3416 - recall_7: 0.6741 - val_loss: 0.4718 - val_recall_7: 0.4829
Epoch 5/50
163/163 [==============================] - 1s 6ms/step - loss: 0.3022 - recall_7: 0.7263 - val_loss: 0.7100 - val_recall_7: 0.2949
Epoch 6/50
163/163 [==============================] - 1s 4ms/step - loss: 0.2964 - recall_7: 0.7218 - val_loss: 0.5260 - val_recall_7: 0.4658
Epoch 7/50
163/163 [==============================] - 1s 4ms/step - loss: 0.2420 - recall_7: 0.7800 - val_loss: 1.4140 - val_recall_7: 0.111

In [71]:
es_cb_first_results_train = es_cb_first_model.evaluate(train_img, train_y)
es_cb_first_results_val = es_cb_first_model.evaluate(val_img, val_y)

print(first_results_train)
print(first_results_val)
print(cb_first_results_train)
print(cb_first_results_val)
print(es_cb_first_results_train)
print(es_cb_first_results_val)

20/20 [==============================] - 0s 2ms/step - loss: 0.6010 - recall_7: 0.4573
[0.07539720833301544, 0.9276659488677979]
[1.0559560060501099, 0.3076923191547394]
[0.09835421293973923, 0.9753915071487427]
[0.8245831727981567, 0.47863247990608215]
[0.11735934764146805, 0.9485458731651306]
[0.6010156273841858, 0.4572649598121643]


In [72]:
#Load the best (saved) model
from keras.models import load_model
saved_model = load_model('best_model.h5')

In [73]:
save_results_train = saved_model.evaluate(train_img, train_y)
save_results_val = saved_model.evaluate(val_img, val_y)

20/20 [==============================] - 0s 3ms/step - loss: 0.4718 - recall_7: 0.4829


Dropout 

In [75]:
model = models.Sequential()
model.add(layers.Dropout(0.3, input_shape=(12288,)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

In [76]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [77]:
result = model.fit(train_img,
                    train_y,
                    epochs=50,
                    class_weight = class_weight,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
163/163 [==============================] - 1s 5ms/step - loss: 0.9526 - recall_8: 0.0037 - val_loss: 0.7893 - val_recall_8: 0.0000e+00
Epoch 2/50
163/163 [==============================] - 1s 5ms/step - loss: 0.7216 - recall_8: 0.0000e+00 - val_loss: 0.6895 - val_recall_8: 0.0000e+00
Epoch 3/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6748 - recall_8: 0.0000e+00 - val_loss: 0.6060 - val_recall_8: 0.0000e+00
Epoch 4/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6389 - recall_8: 0.0000e+00 - val_loss: 0.7531 - val_recall_8: 0.0000e+00
Epoch 5/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6398 - recall_8: 0.0000e+00 - val_loss: 0.8462 - val_recall_8: 0.0000e+00
Epoch 6/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6206 - recall_8: 0.0000e+00 - val_loss: 0.9041 - val_recall_8: 0.0000e+00
Epoch 7/50
163/163 [==============================] - 1s 5ms/step - loss: 0.6174 - recall_8: 0.0000e+00 

L1 Regularization

In [79]:
from keras import regularizers
L1_model = models.Sequential()
L1_model.add(layers.Dense(20, activation='relu', kernel_regularizer=regularizers.l1(0.005), input_shape=(12288,)))
L1_model.add(layers.Dense(1, activation='sigmoid'))

In [81]:
L1_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [82]:
L1_result = L1_model.fit(train_img,
                    train_y,
                    epochs=50,
                    class_weight = class_weight,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/50
163/163 [==============================] - 1s 5ms/step - loss: 10.1437 - recall_10: 0.0172 - val_loss: 6.8807 - val_recall_10: 0.0000e+00
Epoch 2/50
163/163 [==============================] - 1s 4ms/step - loss: 3.7244 - recall_10: 0.1626 - val_loss: 1.8197 - val_recall_10: 0.0299
Epoch 3/50
163/163 [==============================] - 1s 4ms/step - loss: 1.2603 - recall_10: 0.1156 - val_loss: 1.0869 - val_recall_10: 0.0000e+00
Epoch 4/50
163/163 [==============================] - 1s 4ms/step - loss: 0.9407 - recall_10: 0.1193 - val_loss: 1.0097 - val_recall_10: 0.0000e+00
Epoch 5/50
163/163 [==============================] - 1s 4ms/step - loss: 0.9176 - recall_10: 0.1268 - val_loss: 0.8719 - val_recall_10: 0.0000e+00
Epoch 6/50
163/163 [==============================] - 1s 4ms/step - loss: 0.8456 - recall_10: 0.1641 - val_loss: 0.8102 - val_recall_10: 0.0684
Epoch 7/50
163/163 [==============================] - 1s 4ms/step - loss: 0.8575 - recall_10: 0.2446 - val_loss: 1.0830

In [85]:
L1_results_train = L1_model.evaluate(train_img, train_y)
L1_results_val = L1_model.evaluate(val_img, val_y)

print(first_results_train)
print(first_results_val)
print(cb_first_results_train)
print(cb_first_results_val)
print(L1_results_train)
print(L1_results_val)

20/20 [==============================] - 0s 2ms/step - loss: 1.0807 - recall_10: 0.0598
[0.07539720833301544, 0.9276659488677979]
[1.0559560060501099, 0.3076923191547394]
[0.09835421293973923, 0.9753915071487427]
[0.8245831727981567, 0.47863247990608215]
[0.5949401259422302, 0.4168531000614166]
[1.080660343170166, 0.05982905998826027]


# CNN

In [116]:
train_img.shape

(5216, 12288)

In [122]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(12288,)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=[tf.keras.metrics.Recall()])

ValueError: Input 0 of layer conv2d_25 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: [None, 12288]

In [119]:
model_result = model.fit(train_img,
                    train_y,
                    epochs=30,
                    class_weight = class_weight,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Epoch 1/30


ValueError: in user code:

    /Users/jaclynhenn/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/jaclynhenn/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jaclynhenn/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jaclynhenn/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/jaclynhenn/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_

**Second Model**

In [ ]:
second_model = models.Sequential()
second_model.add(layers.Dense(20, activation='relu', input_shape=(12288,)))
second_model.add(layers.Dense(7, activation='relu'))
second_model.add(layers.Dense(5, activation='relu'))
second_model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
second_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall()])

In [ ]:
second_model.summary()

In [ ]:
result_second = second_model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_img, val_y))

Visualize Loss and Recall Scores

In [ ]:
second_results_train = second_model.evaluate(train_img, train_y)
second_results_val = second_model.evaluate(val_img, val_y)

print(first_results_train)
print(first_results_val)
print(second_results_train)
print(second_results_val)

Early Stopping

In [ ]:
# Get few samples for both the classes
pneumonia_samples = (train_data[train_data['label']==1]['image'].iloc[:5]).tolist()
normal_samples = (train_data[train_data['label']==0]['image'].iloc[:5]).tolist()

# Concat the data in a single list and del the above two list
samples = pneumonia_samples + normal_samples
del pneumonia_samples, normal_samples

# Plot the data 
f, ax = plt.subplots(2,5, figsize=(30,10))
for i in range(10):
    img = imread(samples[i])
    ax[i//5, i%5].imshow(img, cmap='gray')
    if i<5:
        ax[i//5, i%5].set_title("Pneumonia")
    else:
        ax[i//5, i%5].set_title("Normal")
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_aspect('auto')
plt.show()